# Imports

In [ ]:
# General includes.
import os
import gc
import random
import threading
import math
import itertools
import functools
from copy import deepcopy
#from termcolor import colored, cprint
import colored
from datetime import datetime, timedelta
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from matplotlib import pyplot as plt

# Typing includes.
from typing import Dict, List, Optional, Any, Tuple

# Numerical includes.
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# pyHealth includes.
from pyhealth.datasets import BaseDataset, MIMIC3Dataset, SampleDataset , split_by_patient
from pyhealth.data import Patient, Visit, Event
from pyhealth.data import Event, Visit, Patient
from pyhealth.datasets.utils import strptime

In [ ]:
# Model imports 
from transformers import BertTokenizer, BertModel, BertForMaskedLM, BertConfig, BertTokenizerFast
from transformers import TensorType
from transformers import AutoConfig, AutoModel

# Globals

In [ ]:
USE_GPU_ = False
BERT_USE_GPU_ = True  # BERT embeddings 
DEV_ = True  # Uses a small subset of MIMIC data: https://pyhealth.readthedocs.io/en/latest/api/datasets/pyhealth.datasets.MIMIC3Dataset.html#pyhealth.datasets.MIMIC3Dataset
GPU_STR_ = 'cuda'
# DATA_DIR_ = os.path.join(os.getcwd(), DATA_DIR_)
DATA_DIR_ = '~/sw/physionet.org/files/mimiciii/1.4'
BATCH_SIZE_ = 32
EMBEDDING_DIM_ = 264  # BERT requires a multiple of 12
SHUFFLE_ = True
SAMPLE_MULTIPLIER_ = 1

# TBD cache for BERT embeddings.
CACHE_DIR_ = 'cache'

# Set seed for reproducibility.
seed = 90210
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

# Preprocessing

### Load MIMIC III Data

In [ ]:
from pyhealth.medcode import InnerMap, ICD9CM

icd9cm = InnerMap.load("ICD9CM")
icd9cm.lookup("428.0") # get detailed info
icd9cm.get_ancestors("428.0") # get parents

print(icd9cm.lookup("78951")) # get detailed info
print(f'78951 ancestors {icd9cm.get_ancestors("78951")}') # get parents

print(icd9cm.lookup("7895")) # get detailed info
print(f'7895 ancestors {icd9cm.get_ancestors("7895")}') # get parents


print(icd9cm.lookup("7894")) # get detailed info
print(f'7894 ancestors {icd9cm.get_ancestors("7894")}') # get parents

print(icd9cm.lookup("78941")) # get detailed info
print(f'78941 ancestors {icd9cm.get_ancestors("78941")}') # get parents

print(ICD9CM.standardize('78951'))
print(ICD9CM.standardize('7895'))

In [ ]:
def _compute_duration_minutes(start_datetime: str, end_datetime: str) -> float:
    '''Return duration in minutes as a float.
    '''
    # MIMIC-III uses the following format: 2146-07-22 00:00:00
    start = datetime.strptime(start_datetime, '%Y-%m-%d %H:%M:%S')
    end = datetime.strptime(end_datetime,   '%Y-%m-%d %H:%M:%S')
    return float((end - start).seconds)

class MIMIC3DatasetWrapper(MIMIC3Dataset):
    ''' Add extra tables to the MIMIC III dataset.
    
      Some of the tables we need like "D_ICD_DIAGNOSES", "D_ITEMS", "D_ICD_PROCEDURES"
      are not supported out of the box. 
      
      This class defines parsing methods to extract text data from these extra tables.
      The text data is generally joined on the PATIENTID, HADMID, ITEMID to match the
      pyHealth Vists class representation.
    '''
   
    # We need to add storage for text-based lookup tables here.
    def __init__(self, *args, **kwargs):
        self._valid_text_tables = ["D_ICD_DIAGNOSES", "D_ITEMS", "D_ICD_PROCEDURES", "D_LABITEMS"]
        self._text_descriptions = {x: {} for x in self._valid_text_tables}
        self._text_luts = {x: {} for x in self._valid_text_tables}
        super().__init__(*args, **kwargs)
    
    def get_all_tables(self) -> List[str]: 
        return list(self._text_descriptions.keys())
        
    def get_text_dict(self, table_name: str) -> Dict[str, Dict[Any, Any]]:
        return self._text_descriptions.get(table_name)
    
    def set_text_lut(self, table_name: str, lut: Dict[Any, Any]) -> None:
        self._text_luts[table_name] = lut
    
    def get_text_lut(self, table_name: str) -> Dict[Any, Any]:
        return self._text_luts[table_name]
    
#     def _validate(self) -> Dict:
#         """Helper function which validates the samples.
#         Will be called in `self.__init__()`.
#         Returns:
#             input_info: Dict, a dict whose keys are the same as the keys in the
#                 samples, and values are the corresponding input information:
#                 - "type": the element type of each key attribute, one of float,
#                     int, str.
#                 - "dim": the list dimension of each key attribute, one of 0, 1, 2, 3.
#                 - "len": the length of the vector, only valid for vector-based
#                     attributes.
#         """
#         """ 1. Check if all samples are of type dict. """
#         assert all(
#             [isinstance(s, dict) for s in self.samples],
#         ), "Each sample should be a dict"
#         keys = self.samples[0].keys()

#         """ 2. Check if all samples have the same keys. """
#         assert all(
#             [set(s.keys()) == set(keys) for s in self.samples]
#         ), "All samples should have the same keys"

#         """ 3. Check if "patient_id" and "visit_id" are in the keys."""
#         assert "patient_id" in keys, "patient_id should be in the keys"
#         assert "visit_id" in keys, "visit_id should be in the keys"
#         return {}

    
    def _add_events_to_patient_dict(
        self,
        patient_dict: Dict[str, Patient],
        group_df: pd.DataFrame,
    ) -> Dict[str, Patient]:
        #TODO(botelho3) Imported from PyHealth Base dataset githubf to
        #support parse_prescription
        """Helper function which adds the events column of a df.groupby object to the patient dict.
        
        Will be called at the end of each `self.parse_[table_name]()` function.
        Args:
            patient_dict: a dict mapping patient_id to `Patient` object.
            group_df: a df.groupby object, having two columns: patient_id and events.
                - the patient_id column is the index of the patient
                - the events column is a list of <Event> objects
        Returns:
            The updated patient dict.
        """
        for _, events in group_df.items():
            for event in events:
                patient_dict = self._add_event_to_patient_dict(patient_dict, event)
        return patient_dict

    
    def parse_prescriptions(self, patients: Dict[str, Patient]) -> Dict[str, Patient]:
        """Helper function which parses PRESCRIPTIONS table.
        
        TODO(botelho3) - we have to override this to include the text fields. The
        prescriptions table does not link to a separate D_ICD_* table in MIMIC-III
        thtat contains text descriptions of the prescription. The text descriptions
        are in the columns of this table. Regular pyHealth ignores these columns. We
        override this method to appent pyHealth Event objects containing the text
        columns to each patient.
        
        Will be called in `self.parse_tables()`
        Docs:
            - PRESCRIPTIONS: https://mimic.mit.edu/docs/iii/tables/prescriptions/
        Args:
            patients: a dict of `Patient` objects indexed by patient_id.
        Returns:
            The updated patients dict.
        """
        table = "PRESCRIPTIONS"
        # read table
        df = pd.read_csv(
            os.path.join(self.root, f"{table}.csv"),
            low_memory=False,
            dtype={"SUBJECT_ID": str, "HADM_ID": str, "NDC": str,
                   "DRUG_TYPE": str, "DRUG": str,
                   "PROD_STRENGTH": str, "ROUTE": str, "ENDDATE": str},
        )
        # drop records of the other patients
        df = df[df["SUBJECT_ID"].isin(patients.keys())]
        # drop rows with missing values
        # df = df.dropna(subset=["SUBJECT_ID", "HADM_ID", "NDC", "DRUG_TYPE",
        #                        "DRUG", "PROD_STRENGTH"])
        df = df.dropna()
        # sort by start date and end date
        df = df.sort_values(
            ["SUBJECT_ID", "HADM_ID", "STARTDATE", "ENDDATE"], ascending=True
        )
        # group by patient and visit
        group_df = df.groupby("SUBJECT_ID")
        
        # parallel unit for prescription (per patient)
        def prescription_unit(p_id, p_info):
            events = []
            for v_id, v_info in p_info.groupby("HADM_ID"):
                zipped = zip(v_info["STARTDATE"], v_info["NDC"], v_info["DRUG_TYPE"],
                             v_info["DRUG"], v_info["PROD_STRENGTH"], v_info["ROUTE"],
                             v_info["ENDDATE"])
                for startdate, code, dtype, dname, dose, route, enddate in zipped:
                    if not type(startdate) == str:
                        startdate = '2142-07-18 00:00:00'
                    if not type(enddate) == str:
                        enddate = '2142-07-18 00:00:00'
                    assert(type(dname) is str)
                    # if not type(enddate) is str:
                    #     print(f'Not matching enddate {enddate} startdate {startdate}')
                    #     print(f'dname {dname}, hadm_id {v_id}, p_id {p_id}')
                    assert(type(startdate) is str)
                    assert(type(enddate) is str)
                    event = Event(
                        code=code,
                        table=table,
                        vocabulary="NDC",
                        visit_id=v_id,
                        patient_id=p_id,
                        timestamp=strptime(startdate),
                        dtype=dtype,
                        dname=dname,
                        dose=dose,
                        route=route,
                        duration=_compute_duration_minutes(startdate, enddate),
                    )
                    events.append(event)
            return events

                # parallel apply
        group_df = group_df.parallel_apply(
            lambda x: prescription_unit(x.SUBJECT_ID.unique()[0], x)
        )

        # summarize the results
        # print(f'BASE {dir(BaseDataset)}')
        # print(f'MIMIC {dir(MIMIC3Dataset)}')
        # print(f'WRAPPER {dir(self)}')
        # # patients = BaseDataset._add_events_to_patient_dict(patients, group_df)
        # # patients = self._add_events_to_patient_dict(patients, group_df)
        patients = self._add_events_to_patient_dict(patients, group_df)
        return patients
    
    # Note the name has to match the table name exactly.
    # See https://github.com/sunlabuiuc/PyHealth/blob/master/pyhealth/datasets/mimic3.py#L71.
    def parse_d_icd_diagnoses(self, patients: Dict[str, Patient]) -> Dict[str, Patient]: 
        """Helper function which parses D_ICD_DIAGNOSIS table.
        Will be called in `self.parse_tables()`
        Docs:
            - D_ICD_DIAGNOSIS: https://mimic.mit.edu/docs/iii/tables/d_icd_diagnoses/
        Args:
            patients: a dict of `Patient` objects indexed by patient_id.
        Returns:
            The unchanged patients dict.
        Note:
            This function doesn't up date the patients dict like other part_*() functions.
            Here we read the D_ICD_DIAGNOSES.csv file containing ICD9_code -> text mappings
            and store them in a dict `self._text_descriptions[table]`.
            
            The dict is used as a ICD9_code -> text diagnosis description lookup
            for DescEmb.
        """
        table = "D_ICD_DIAGNOSES"
        print(f"Parsing {table}")
        assert(table in self._valid_text_tables)
        
        
        # read table
        df = pd.read_csv(
            os.path.join(self.root, f"{table}.csv"),
            usecols=["ICD9_CODE", "SHORT_TITLE", "LONG_TITLE"],
            dtype={"ICD9_CODE": str, "SHORT_TITLE": str, "LONG_TITLE": str}
        )
        
        # drop rows with missing values
        df = df.dropna(subset=["ICD9_CODE", "SHORT_TITLE", "LONG_TITLE"])
        # sort by sequence number (i.e., priority)
        df = df.sort_values(["ICD9_CODE"], ascending=True)
       
        # print(df.head())
        self._text_descriptions[table] = df.reset_index(drop=True).to_dict(orient='split')
        
        # We haven't altered the patients array, just return it.
        return patients
    
    def parse_d_labitems(self, patients: Dict[str, Patient]) -> Dict[str, Patient]: 
        """Helper function which parses D_LABITEMS table.
        Will be called in `self.parse_tables()`
        Docs:
            - D_LABITEMS: https://mimic.mit.edu/docs/iii/tables/d_labitems/
        Args:
            patients: a dict of `Patient` objects indexed by patient_id.
        Returns:
            The unchanged patients dict.
        Note:
            This function doesn't up date the patients dict like other part_*() functions.
            Here we read the D_LABITEMS.csv file containing ICD9_code -> text mappings
            and store them in a dict `self._text_descriptions[table]`.
            
            The dict is used as a ICD9_code -> text lab measurement description lookup
            for DescEmb.
        """
        table = "D_LABITEMS"
        print(f"Parsing {table}")
        assert(table in self._valid_text_tables)
        
        # read table
        df = pd.read_csv(
            os.path.join(self.root, f"{table}.csv"),
            usecols=["ITEMID", "LABEL", "CATEGORY", "FLUID"],
            dtype={"ITEMID": str, "LABEL": str, "CATEGORY": str, "FLUID": str}
        )
        
        # drop rows with missing values
        df = df.dropna(subset=["ITEMID", "LABEL", "CATEGORY", "FLUID"])
        # sort by sequence number (i.e., priority)
        df = df.sort_values(["ITEMID"], ascending=True)
       
        self._text_descriptions[table] = df.reset_index(drop=True).to_dict(orient='split')
        
        # We haven't altered the patients array, just return it.
        return patients
    
    
    def parse_d_items(self, patients: Dict[str, Patient]) -> Dict[str, Patient]: 
        # TODO(botelho3) - Note this may not be totally useable because the ITEMID
        # uinqiue key only links to these tables using ITEMID
        #   - INPUTEVENTS_MV 
        #   - OUTPUTEVENTS on ITEMID
        #   - PROCEDUREEVENTS_MV on ITEMID
        # 
        # Not to the tables we want e.g. 
        """Helper function which parses D_ITEMS table.
        Will be called in `self.parse_tables()`
        Docs:
            - D_ITEMS: https://mimic.mit.edu/docs/iii/tables/d_items/
        Args:
            patients: a dict of `Patient` objects indexed by patient_id.
        Returns:
            The unchanged patients dict.
        Note:
            This function doesn't up date the patients dict like other part_*() functions.
            Here we read the D_ITEMS.csv file containing ICD9_code -> text mappings
            and store them in a dict `self._text_descriptions[table]`.
            
            The dict is used as a ICD9_code -> text inputs/output/procedure events lookup
            for DescEmb.
        """
        table = "D_ITEMS"
        print(f"Parsing {table}")
        assert(table in self._valid_text_tables)
        
        # read table
        df = pd.read_csv(
            os.path.join(self.root, f"{table}.csv"),
            usecols=["ITEMID", "LABEL", "CATEGORY"],
            dtype={"ITEMID": str, "LABEL": str, "CATEGORY": str}
        )
        
        # drop rows with missing values
        df = df.dropna(subset=["ITEMID", "LABEL", "CATEGORY"])
        # sort by sequence number (i.e., priority)
        df = df.sort_values(["ITEMID"], ascending=True)
       
        self._text_descriptions[table] = df.reset_index(drop=True).to_dict(orient='split')
        
        # We haven't altered the patients array, just return it.
        return patients
    
    
    def parse_d_icd_procedures(self, patients: Dict[str, Patient]) -> Dict[str, Patient]: 
        """Helper function which parses D_ICD_PROCEDURES table.
        
        Will be called in `self.parse_tables()`
        Docs:
            - D_ICD_PROCEDURES: https://mimic.mit.edu/docs/iii/tables/d_icd_procedures/
        Args:
            patients: a dict of `Patient` objects indexed by patient_id.
        Returns:
            The unchanged patients dict.
        Note:
            This function doesn't up date the patients dict like other part_*() functions.
            Here we read the D_ICD_PROCEDURES.csv file containing ICD9_code -> text mappings
            and store them in a dict `self._text_descriptions[table]`.
            
            The dict is used as a ICD9_code -> text procedure description lookup for DescEmb.
        """
        table = "D_ICD_PROCEDURES"
        print(f"Parsing {table}")
        assert(table in self._valid_text_tables)
        
        # read table
        df = pd.read_csv(
            os.path.join(self.root, f"{table}.csv"),
            usecols=["ICD9_CODE", "SHORT_TITLE", "LONG_TITLE"],
            dtype={"ICD9_CODE": str, "SHORT_TITLE": str, "LONG_TITLE": str}
        )
        
        # drop rows with missing values
        df = df.dropna(subset=["ICD9_CODE", "SHORT_TITLE", "LONG_TITLE"])
        # sort by sequence number (i.e., priority)
        df = df.sort_values(["ICD9_CODE"], ascending=True)
       
        # print(df.head())
        self._text_descriptions[table] = df.reset_index(drop=True).to_dict(orient='split')
        
        # We haven't altered the patients array, just return it.
        return patients
    

In [ ]:
print(f'Reading data from: `{DATA_DIR_}`')

mimic3base = MIMIC3DatasetWrapper(
    # root="https://storage.googleapis.com/pyhealth/Synthetic_MIMIC-III/",
    root=DATA_DIR_,
    dataset_name='mimic_3_dataset',
    tables=["D_ICD_DIAGNOSES", "D_ICD_PROCEDURES", "D_ITEMS", "D_LABITEMS",
            "DIAGNOSES_ICD", "PROCEDURES_ICD", "PRESCRIPTIONS"], # "LABEVENTS"],
    # map all NDC codes to ATC 3-rd level codes in these tables
    # See https://en.wikipedia.org/wiki/Anatomical_Therapeutic_Chemical_Classification_System.
    code_mapping={"NDC": ("ATC", {"target_kwargs": {"level": 3}})},
    # Reads a subset of the data. Disable for full training run.
    dev = DEV_,
    # Slow, rebuilds the dataset instead of caching.
    refresh_cache=True,
)

mimic3base.stat()
mimic3base.info()


In [ ]:
table_names = mimic3base.get_all_tables()
print(table_names)

print('\033[92m' '====Tables====\n' '\033[0m')
# print(colored('====Tables====\n', 'green'))
# print(colored.fg('green') + '====Tables====\n')
for t in table_names:
    d = mimic3base.get_text_dict(t)
    print(f"Table: {t}")
    print(d['data'][:5])
    print('\n\n')

# Take the cached tables from the parse_tables function and build the {ICD9 -> (short_name, long_name)}
# lookup tables.
for t in table_names:
    d = mimic3base.get_text_dict(t)
    d = d['data']
    lut = {record[0]: record[1:] for record in d}
    mimic3base.set_text_lut(t,  lut)
    
print('\033[92m' '====Luts====\n' '\033[0m')
# print(f'{colored.fg("green")} ====Luts====\n')
for t in table_names:
    d = mimic3base.get_text_lut(t)
    print(f"Lut {t}:\n{dict(itertools.islice(d.items(), 2))}")

### Tasks

Declare tasks for 2 of the 5 prediction tasks specified in the paper. We will create dataloaders for each task that contain the ICD codes and the raw text for each (patient, visit).

#### CodeEMB Pred tasks

In [ ]:
# The original authors tackled 5 tasks
#   1. readmission
#   2. mortality
#   3. an ICU stay exceeding three days
#   4. an ICU stay exceeding seven days
#   5. diagnosis prediction

def readmission_pred_task(patient, time_window=3):
    """
    patient is a <pyhealth.data.Patient> object
    """
    samples = []

    # we will drop the last visit
    for i in range(len(patient) - 1):
        visit: Visit = patient[i]
        next_visit: Visit = patient[i + 1]

        # step 1: define the readmission label 
        # get time difference between current visit and next visit
        time_diff = (next_visit.encounter_time - visit.encounter_time).days
        readmission_label = 1 if time_diff < time_window else 0
        
        # step 2: get code-based feature information
        conditions = visit.get_code_list(table="DIAGNOSES_ICD")
        procedures = visit.get_code_list(table="PROCEDURES_ICD")
        drugs = visit.get_code_list(table="PRESCRIPTIONS")
        drugs_full = visit.get_event_list(table="PRESCRIPTIONS")
        
        # step 3: exclusion criteria: visits without condition, procedure, or drug
        if len(conditions) * len(procedures) == 0 * len(drugs_full) == 0:
            continue
        if len(conditions) + len(procedures) + len(drugs_full) < 5:
            # Exclude stays with less than 5 procedures.
            continue
        
        # step 3.5: build text lists from the ICD codes
        diag_lut = mimic3base.get_text_lut("D_ICD_DIAGNOSES")
        proc_lut = mimic3base.get_text_lut("D_ICD_PROCEDURES")
        conditions_text = [diag_lut.get(cond,("", ""))[1] for cond in conditions]
        procedures_text = [proc_lut.get(proc,("", ""))[1] for proc in procedures]
        drugs_text = [' '.join([d['dname'], d['dtype'], d['dose'], d['route'], str(d['duration'])])
                      for d in [d.attr_dict for d in drugs_full]]
            
        # step 4: assemble the samples
        # TODO: should also exclude visit with age < 18
        samples.append(
            {
                "visit_id": visit.visit_id,
                "patient_id": patient.patient_id,
                # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
                "conditions": conditions,
                "procedures": procedures,
                "conditions_text": conditions_text,
                "procedures_text": procedures_text,
                "drugs": drugs,
                "drugs_text": drugs_text,
                # "labevents": labevents,
                # "labevents_text": labevents_text
                "label": readmission_label,
            }
        )
    # no cohort selection
    return samples


def mortality_pred_task(MORTALITY_PER_VISIT_ICD_9_CODE_COUNT_, patient):
    """
    patient is a <pyhealth.data.Patient> object
    """
    samples = []

    # loop over all visits but the last one
    for i in range(len(patient) - 1):

        # visit and next_visit are both <pyhealth.data.Visit> objects
        # there are no vists.attr_dict keys
        visit = patient[i]
        next_visit = patient[i + 1]
        # if i == 0: print(visit)

        # step 1: define the mortality_label
        if next_visit.discharge_status not in [0, 1]:
            mortality_label = 0
        else:
            mortality_label = int(next_visit.discharge_status)

        # step 2: get code-based feature information
        conditions = visit.get_code_list(table="DIAGNOSES_ICD")
        procedures = visit.get_code_list(table="PROCEDURES_ICD")
        drugs = visit.get_code_list(table="PRESCRIPTIONS")
        drugs_full = visit.get_event_list(table="PRESCRIPTIONS")
        # if i == 0: print([d.attr_dict for d in drugs_full])
        # if i == 0: print(conditions)
        # if i == 0: print(procedures)
        # if i == 0: print(drugs)
        # TODO(botelho3) - add this datasource back in once we have full MIMIC-III dataset.
        # labevents = visit.get_code_list(table="LABEVENTS")

        # step 3: exclusion criteria: visits without condition, procedure, or drug
        if len(conditions) * len(procedures) == 0 * len(drugs_full) == 0:
            continue
        if len(conditions) + len(procedures) + len(drugs_full) < 5:
            # Exclude stays with less than 5 procedures.
            continue
        
        # step 3.5: build text lists from the ICD codes
        diag_lut = mimic3base.get_text_lut("D_ICD_DIAGNOSES")
        proc_lut = mimic3base.get_text_lut("D_ICD_PROCEDURES")
        # if i == 0: print(diag_lut)
        # if i == 0: print(proc_lut)
        # Index 0 is shortname, index 1 is longname.
        # print([str(cond) + ' ' + str(diag_lut.get(cond)) for cond in conditions])
        # print(proc_lut.get(procedures[0]))
        conditions_text = [diag_lut.get(cond,("", ""))[1] for cond in conditions]
        procedures_text = [proc_lut.get(proc,("", ""))[1] for proc in procedures]
        drugs_text = [' '.join([d['dname'], d['dtype'], d['dose'], d['route'], str(d['duration'])])
                      for d in [d.attr_dict for d in drugs_full]]
        # TODO(botelho3) - add the labevents data source back in once we have full MIMIC-III dataset.
        # labevents_text =
        
        # step 4: assemble the samples
        samples.append(
            {
                "visit_id": visit.visit_id,
                "patient_id": patient.patient_id,
                # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
                "conditions": conditions,
                "procedures": procedures,
                "conditions_text": conditions_text,
                "procedures_text": procedures_text,
                "drugs": drugs,
                "drugs_text": drugs_text,
                # "labevents": labevents,
                # "labevents_text": labevents_text
                "label": mortality_label,
            }
        )
    return samples

def mortality_pred_task_cemb(CODE_COUNT, patient):
    """
    patient is a <pyhealth.data.Patient> object
    each sample is a list of vists for 1 patient.
    """
   
    # TODO(botelho3) - stupid hack around the limitations of SampleDataset validator.
    kMaxVisits = 5
    if len(patient) < 1 or len(patient) > kMaxVisits:
        return []
        
    samples = [{
        "visit_id": patient[0].visit_id,
        "patient_id": patient.patient_id,
        # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
        "num_visits": 0,
        "conditions": [[] for v in range(kMaxVisits)],
        "procedures": [[] for v in range(kMaxVisits)],
        "conditions_text": [[] for v in range(kMaxVisits)],
        "procedures_text": [[] for v in range(kMaxVisits)],
        "drugs": [[] for v in range(kMaxVisits)],
        "drugs_text": [[] for v in range(kMaxVisits)],
        # "labevents": labevents,
        # "labevents_text": labevents_text
        "label": 0,
    }]
    
    global_mortality_label = 0
    # loop over all visits but the last one
    for i in range(len(patient)):

        # visit and next_visit are both <pyhealth.data.Visit> objects
        # there are no vists.attr_dict keys
        visit: Visit = patient[i]
        # next_visit: Visit = patient[i + 1]
        # if i == 0: print(visit)

        # step 1: define the mortality_label
        # if next_visit.discharge_status not in [0, 1]:
        #     mortality_label = 0
        # else:
        #     mortality_label = int(next_visit.discharge_status)
        mortality_label = int(visit.discharge_status)
        global_mortality_label |= mortality_label

    # loop over all visits
    out_idx = 0
    for i, visit in enumerate(patient):
        # visit and next_visit are both <pyhealth.data.Visit> objects
        # there are no vists.attr_dict keys

        # step 2: get code-based feature information
        conditions = visit.get_code_list(table="DIAGNOSES_ICD")
        procedures = visit.get_code_list(table="PROCEDURES_ICD")
        drugs = visit.get_code_list(table="PRESCRIPTIONS")
        drugs_full = visit.get_event_list(table="PRESCRIPTIONS")
        # if i == 0: print([d.attr_dict for d in drugs_full])
        # if i == 0: print(conditions)
        # if i == 0: print(procedures)
        # if i == 0: print(drugs)
        # TODO(botelho3) - add this datasource back in once we have full MIMIC-III dataset.
        # labevents = visit.get_code_list(table="LABEVENTS")

        # step 3: exclusion criteria: visits without condition, procedure, or drug
        if len(conditions) * len(procedures) * len(drugs_full) == 0:
            continue
        if len(conditions) + len(procedures) + len(drugs_full) < 5:
            # Exclude stays with less than 5 procedures.
            continue
        
        # step 3.5: build text lists from the ICD codes
        diag_lut = mimic3base.get_text_lut("D_ICD_DIAGNOSES")
        proc_lut = mimic3base.get_text_lut("D_ICD_PROCEDURES")
        # if i == 0: print(diag_lut)
        # if i == 0: print(proc_lut)
        # Index 0 is shortname, index 1 is longname.
        # print([str(cond) + ' ' + str(diag_lut.get(cond)) for cond in conditions])
        # print(proc_lut.get(procedures[0]))
        conditions_text = [diag_lut.get(cond,("", ""))[1] for cond in conditions]
        procedures_text = [proc_lut.get(proc,("", ""))[1] for proc in procedures]
        drugs_text = [' '.join([d['dname'], d['dtype'], str(d['dose']), d['route'], str(d['duration'])])
                      for d in [d.attr_dict for d in drugs_full]]
        # TODO(botelho3) - add the labevents data source back in once we have full MIMIC-III dataset.
        # labevents_text =
        
        # step 4: assemble the samples
        # Each field is a list of visits.
        samples[0]['num_visits'] = samples[0]['num_visits'] + 1
        samples[0]['conditions'][out_idx]=conditions
        samples[0]['procedures'][out_idx]=procedures
        samples[0]['conditions_text'][out_idx]= conditions_text
        samples[0]['procedures_text'][out_idx] = procedures_text
        samples[0]['drugs'][out_idx] = drugs
        samples[0]['drugs_text'][out_idx] = drugs_text
        out_idx = out_idx + 1
    samples[0]['label'] = global_mortality_label
   
    # Record all unique codes and their frequency for LUT.
    for visit in samples[0]['conditions']:
        for code in visit:
            CODE_COUNT[code] = CODE_COUNT.get(code, 0) + 1
           
    # If none of the samples met the criteria return an empty list.
    if samples[0]['num_visits'] == 0:
        return []

    # Potentially multiply the sample n-times to increase dataset size.
    samples.extend(
        list(deepcopy(samples[0]) for s in range(SAMPLE_MULTIPLIER_-1))
    )
        
    return samples


#### DescEmb Pred Tasks

In [ ]:
# The original authors tackled 5 tasks
#   1. readmission
#   2. mortality
#   3. an ICU stay exceeding three days
#   4. an ICU stay exceeding seven days
#   5. diagnosis prediction

READMISSION_PER_PATIENT_ICD_9_CODE_COUNT_ = {}
READMISSION_PER_PATIENT_ICD_9_CODE2IDX_ = {}
#{code: idx for idx, code in enumerate(READMISSION_PER_PATIENT_ICD_9_CODE_COUNT_.keys())}
def readmission_pred_task_per_patient(patient, time_window=3):
    """
    patient is a <pyhealth.data.Patient> object
    """
    samples = []
    visits = []
    kMaxListSize = 40
   
    # Length 1 patients by defn are not readmitted.
    if len(patient) < 1:
        return samples

    # we will drop the last visit
    global_readmission_label = 0
    for i in range(len(patient) - 1):
        visit: Visit = patient[i]
        next_visit: Visit = patient[i + 1]

        # step 1: define the readmission label 
        # get time difference between current visit and next visit
        time_diff = (next_visit.encounter_time - visit.encounter_time).days
        readmission_label = 1 if time_diff < time_window else 0
        global_readmission_label |= readmission_label  
       
    for i in range(len(patient)):
        visit: Visit = patient[i]
        # step 2: get code-based feature information
        conditions = visit.get_code_list(table="DIAGNOSES_ICD")
        procedures = visit.get_code_list(table="PROCEDURES_ICD")
        # drugs = visit.get_code_list(table="PRESCRIPTIONS")
        drugs = [x.code for x in visit.get_event_list(table="PRESCRIPTIONS")]
        drugs_full = visit.get_event_list(table="PRESCRIPTIONS")
        
        # step 3: exclusion criteria: visits without condition, procedure, or drug
        if len(conditions) * len(procedures) == 0 * len(drugs_full) == 0:
            continue
        if len(conditions) + len(procedures) + len(drugs_full) < 5:
            # Exclude stays with less than 5 procedures.
            continue
        
        # step 3.5: build text lists from the ICD codes
        d_diag = mimic3base.get_text_lut("D_ICD_DIAGNOSES")
        d_proc = mimic3base.get_text_lut("D_ICD_PROCEDURES")
        conditions_text = [d_diag.get(cond,("", ""))[1] for cond in conditions]
        procedures_text = [d_proc.get(proc,("", ""))[1] for proc in procedures]
        drugs_text = [' '.join([d['dname'], d['dtype'], d['dose'], d['route'], str(d['duration'])])
                      for d in [d.attr_dict for d in drugs_full]]
            
        # step 4: assemble the samples
        visits.append(
            {
                "visit_id": visit.visit_id,
                "patient_id": patient.patient_id,
                # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
                "conditions": conditions,
                "procedures": procedures,
                "conditions_text": conditions_text,
                "procedures_text": procedures_text,
                "drugs": drugs,
                "drugs_text": drugs_text,
                # "labevents": labevents,
                # "labevents_text": labevents_text
                "label": global_readmission_label,
            }
        )
        

    # Return empty list, didn't meet exclusion criteria.
    num_visits = len(visits)
    if num_visits < 1:
        return samples
    
    def pad_field(field, visits, empty_val: Any):
        l = [empty_val for x in range(kMaxListSize)]
        data = [x[field] for x in visits]
        data = list(itertools.chain.from_iterable(data))
        slice_size = min(kMaxListSize, len(data)) - 1
        l[:slice_size] = data[:slice_size]
        return l, slice_size 
    
    conditions, conditions_pad = pad_field("conditions", visits, '0')
    conditions_text, conditions_text_pad = pad_field("conditions_text", visits, '')
    procedures, procedures_pad = pad_field("procedures", visits, '0')
    procedures_text, procedures_text_pad = pad_field("procedures_text", visits, '')
    drugs, drugs_pad = pad_field("drugs", visits, '0')
    drugs_text, drugs_text_pad = pad_field("drugs_text", visits, '')
    assert(drugs_pad == drugs_text_pad)
    sample = {
        "patient_id": patient.patient_id,
        # TODO(botelho3) Why does pyhealth require a visit id in the keys if we're combining vists?
        "visit_id": visits[0]["visit_id"],
        "num_visits": num_visits,
        # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
        "conditions": conditions,
        "conditions_text": conditions_text,
        "procedures": procedures,
        "procedures_text": procedures_text,
        "drugs": drugs,
        "drugs_text": drugs_text,
        
        "conditions_pad": conditions_pad,
        "procedures_pad": procedures_pad,
        "conditions_text_pad": conditions_text_pad,
        "procedures_text_pad": procedures_text_pad,
        "drugs_pad": drugs_pad,
        "drugs_text_pad": drugs_text_pad,
        # "labevents": labevents,
        # "labevents_text": labevents_text
        "label": global_readmission_label,
    }
    # sample = {
    #     "patient_id": patient.patient_id,
    #     # TODO(botelho3) Why does pyhealth require a visit id in the keys if we're combining vists?
    #     "visit_id": visits[0]["visit_id"],
    #     "num_visits": num_visits,
    #     # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
    #     "conditions": [[x["conditions"] for x in visits]],
    #     "procedures": [[x["procedures"] for x in visits]],
    #     "conditions_text": [[x["conditions_text"] for x in visits]],
    #     "procedures_text": [[x["procedures_text"] for x in visits]],
    #     "drugs": [[[x["drugs"] for x in visits]]],
    #     "drugs_text": [[x["drugs_text"] for x in visits]],
    #     # "labevents": labevents,
    #     # "labevents_text": labevents_text
    #     "label": global_readmission_label,
    # }
    for code in sample['conditions']:
        READMISSION_PER_PATIENT_ICD_9_CODE_COUNT_[code] = READMISSION_PER_PATIENT_ICD_9_CODE_COUNT_.get(code, 0) + 1

    samples.extend(
        list(deepcopy(sample) for s in range(SAMPLE_MULTIPLIER_))
    )
    return samples


def mortality_pred_task_per_patient(CODE_COUNT, patient):
    """
    patient is a <pyhealth.data.Patient> object
    """
    samples = []
    visits = []
    kMaxListSize = 40

    global_mortality_label = 0
    # loop over all visits but the last one
    for i in range(len(patient)):

        # visit and next_visit are both <pyhealth.data.Visit> objects
        # there are no vists.attr_dict keys
        visit: Visit = patient[i]
        # next_visit: Visit = patient[i + 1]
        # if i == 0: print(visit)

        # step 1: define the mortality_label
        # if next_visit.discharge_status not in [0, 1]:
        #     mortality_label = 0
        # else:
        #     mortality_label = int(next_visit.discharge_status)
        mortality_label = int(visit.discharge_status)
        global_mortality_label |= mortality_label
        
    # loop over all visits but the last one
    for i, visit in enumerate(patient):
        # visit: Visit.
        
        # step 2: get code-based feature information
        conditions = visit.get_code_list(table="DIAGNOSES_ICD")
        procedures = visit.get_code_list(table="PROCEDURES_ICD")
        # drugs = visit.get_code_list(table="PRESCRIPTIONS")
        drugs = [x.code for x in visit.get_event_list(table="PRESCRIPTIONS")]
        drugs_full = visit.get_event_list(table="PRESCRIPTIONS")
        # if i == 0: print([d.attr_dict for d in drugs_full])
        # if i == 0: print(conditions)
        # if i == 0: print(procedures)
        # if i == 0: print(drugs)
        # TODO(botelho3) - add this datasource back in once we have full MIMIC-III dataset.
        # labevents = visit.get_code_list(table="LABEVENTS")

        # step 3: exclusion criteria: visits without condition, procedure, or drug
        if len(conditions) * len(procedures) == 0 * len(drugs_full) == 0:
            continue
        if len(conditions) + len(procedures) + len(drugs_full) < 5:
            # Exclude stays with less than 5 procedures.
            continue
        
        # step 3.5: build text lists from the ICD codes
        diag_lut = mimic3base.get_text_lut("D_ICD_DIAGNOSES")
        proc_lut = mimic3base.get_text_lut("D_ICD_PROCEDURES")
        # if i == 0: print(d_diag)
        # if i == 0: print(d_proc)
        # Index 0 is shortname, index 1 is longname.
        # print([str(cond) + ' ' + str(d_diag.get(cond)) for cond in conditions])
        # print(d_proc.get(procedures[0]))
        conditions_text = [diag_lut.get(cond,("", ""))[1] for cond in conditions]
        procedures_text = [proc_lut.get(proc,("", ""))[1] for proc in procedures]
        drugs_text = [' '.join([d['dname'], d['dtype'], d['dose'], d['route'], str(d['duration'])])
                      for d in [d.attr_dict for d in drugs_full]]
        # TODO(botelho3) - add the labevents data source back in once we have full MIMIC-III dataset.
        # labevents_text =
        
        # step 4: assemble the samples into a pyHealth Visit.
        visits.append(
            {
                "visit_id": visit.visit_id,
                "patient_id": patient.patient_id,
                # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
                "conditions": conditions,
                "procedures": procedures,
                "conditions_text": conditions_text,
                "procedures_text": procedures_text,
                "drugs": drugs,
                "drugs_text": drugs_text,
                # "labevents": labevents,
                # "labevents_text": labevents_text
                "label": global_mortality_label,
            }
        )
    
    # Return empty list, didn't meet exclusion criteria.
    num_visits = len(visits)
    if num_visits < 1:
        return [] 
   
    # pyHealth requires that all list fields in sample are equal size.
    def pad_field(field, visits, empty_val: Any):
        l = [empty_val for x in range(kMaxListSize)]
        data = [x[field] for x in visits]
        data = list(itertools.chain.from_iterable(data))
        slice_size = min(kMaxListSize, len(data))
        l[:slice_size] = data[:slice_size]
        return l, slice_size
    
    conditions, conditions_pad = pad_field("conditions", visits, '0')
    conditions_text, conditions_text_pad = pad_field("conditions_text", visits, '')
    procedures, procedures_pad = pad_field("procedures", visits, '0')
    procedures_text, procedures_text_pad = pad_field("procedures_text", visits, '')
    drugs, drugs_pad = pad_field("drugs", visits, '0')
    drugs_text, drugs_text_pad = pad_field("drugs_text", visits, '')
    assert(drugs_pad == drugs_text_pad)
    sample = {
        "patient_id": patient.patient_id,
        # TODO(botelho3) Why does pyhealth require a visit id in the keys if we're combining vists?
        "visit_id": visits[0]["visit_id"],
        "num_visits": num_visits,
        # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
        "conditions": conditions,
        "conditions_text": conditions_text,
        "procedures": procedures,
        "procedures_text": procedures_text,
        "drugs": drugs,
        "drugs_text": drugs_text,
        
        "conditions_pad": conditions_pad,
        "procedures_pad": procedures_pad,
        "conditions_text_pad": conditions_text_pad,
        "procedures_text_pad": procedures_text_pad,
        "drugs_pad": drugs_pad,
        "drugs_text_pad": drugs_text_pad,
        # "labevents": labevents,
        # "labevents_text": labevents_text
        "label": global_mortality_label,
    }
    # sample = {
    #     "patient_id": patient.patient_id,
    #     # TODO(botelho3) Why does pyhealth require a visit id in the keys if we're combining vists?
    #     "visit_id": visits[0]["visit_id"],
    #     "num_visits": num_visits,
    #     # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
    #     "conditions": [[x["conditions"] for x in visits]],
    #     "procedures": [[x["procedures"] for x in visits]],
    #     "conditions_text": [[x["conditions_text"] for x in visits]],
    #     "procedures_text": [[x["procedures_text"] for x in visits]],
    #     "drugs": [[[x["drugs"] for x in visits]]],
    #     "drugs_text": [[x["drugs_text"] for x in visits]],
    #     # "labevents": labevents,
    #     # "labevents_text": labevents_text
    #     "label": global_mortality_label,
    # }
   
   
    # For every condition in the sample (all visits). Record frequency.
    # Will be used to build code->index LUT.
    for code in sample['conditions']:
        CODE_COUNT[code] = CODE_COUNT.get(code, 0) + 1

    if SAMPLE_MULTIPLIER_:
        samples.extend(
            list(deepcopy(sample) for s in range(SAMPLE_MULTIPLIER_))
        )
        
    return samples

#### Test Load Readmission Dataset

In [ ]:
# set_task() returns a SampleEHRDataset object
readm_dataset = mimic3base.set_task(readmission_pred_task_per_patient)
READMISSION_PER_PATIENT_ICD_9_CODE2IDX_ = {
    code: idx for idx, code in enumerate(sorted(READMISSION_PER_PATIENT_ICD_9_CODE_COUNT_.keys()))
}
readm_dataset.stat()
readm_dataset.samples[1]
# TODO(botelho3) could try a freq codes limit on this.
print(f"READMISSION_PER_PATIENT_ICD_9_CODE2IDX_ len: {len(READMISSION_PER_PATIENT_ICD_9_CODE2IDX_)}\n"
      f"{READMISSION_PER_PATIENT_ICD_9_CODE2IDX_}")

#### Test Load Mortality Dataset

In [ ]:
MORTALITY_PER_PATIENT_ICD_9_CODE_COUNT_ = {}
MORTALITY_PER_PATIENT_ICD_9_CODE2IDX_ = {}
task_fn = functools.partial(mortality_pred_task_per_patient, MORTALITY_PER_PATIENT_ICD_9_CODE_COUNT_)
mor_dataset = mimic3base.set_task(task_fn, task_name=mortality_pred_task_per_patient.__name__)
MORTALITY_PER_PATIENT_ICD_9_CODE2IDX_ = {
    code: idx for idx, code in enumerate(sorted(MORTALITY_PER_PATIENT_ICD_9_CODE_COUNT_.keys()))
}
mor_dataset.stat()
mor_dataset.samples[1]
# TODO(botelho3) could try a freq codes limit on this.
print(f"MORTALITY_PER_PATIENT_ICD_9_CODE2IDX_ len: {len(MORTALITY_PER_PATIENT_ICD_9_CODE2IDX_)}\n"
      f"{MORTALITY_PER_PATIENT_ICD_9_CODE2IDX_}")

In [ ]:
# The original authors tackled 5 tasks
#   1. readmission
#   2. mortality
#   3. an ICU stay exceeding three days
#   4. an ICU stay exceeding seven days
#   5. diagnosis prediction

def readmission_pred_no_visit_task(patient, time_window=3):
    """
    patient is a <pyhealth.data.Patient> object
    """
    samples = []

    # we will drop the last visit
    for i in range(len(patient) - 1):
        visit: Visit = patient[i]
        next_visit: Visit = patient[i + 1]

        # step 1: define the readmission label 
        # get time difference between current visit and next visit
        time_diff = (next_visit.encounter_time - visit.encounter_time).days
        readmission_label = 1 if time_diff < time_window else 0
        
        # step 2: get code-based feature information
        conditions = visit.get_code_list(table="DIAGNOSES_ICD")
        procedures = visit.get_code_list(table="PROCEDURES_ICD")
        drugs = visit.get_code_list(table="PRESCRIPTIONS")
        drugs_full = visit.get_event_list(table="PRESCRIPTIONS")
        
        # step 3: exclusion criteria: visits without condition, procedure, or drug
        if len(conditions) * len(procedures) == 0 * len(drugs_full) == 0:
            continue
        
        # step 3.5: build text lists from the ICD codes
        d_diag = mimic3base.get_text_lut("D_ICD_DIAGNOSES")
        d_proc = mimic3base.get_text_lut("D_ICD_PROCEDURES")
        conditions_text = [d_diag.get(cond,("", ""))[1] for cond in conditions]
        procedures_text = [d_proc.get(proc,("", ""))[1] for proc in procedures]
        drugs_text = [' '.join([d['dname'], d['dtype'], d['dose'], d['route'], str(d['duration'])])
                      for d in [d.attr_dict for d in drugs_full]]
            
        # step 4: assemble the samples
        # TODO: should also exclude visit with age < 18
        samples.append(
            {
                "visit_id": visit.visit_id,
                "patient_id": patient.patient_id,
                # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
                "conditions": conditions,
                "procedures": procedures,
                "conditions_text": conditions_text,
                "procedures_text": procedures_text,
                "drugs": drugs,
                "drugs_text": drugs_text,
                # "labevents": labevents,
                # "labevents_text": labevents_text
                "label": readmission_label,
            }
        )
    # no cohort selection
    return samples


def mortality_pred_no_visit_task(patient):
    """
    patient is a <pyhealth.data.Patient> object
    """
    samples = []

    # loop over all visits but the last one
    for i in range(len(patient) - 1):

        # visit and next_visit are both <pyhealth.data.Visit> objects
        # there are no vists.attr_dict keys
        visit = patient[i]
        next_visit = patient[i + 1]
        # if i == 0: print(visit)

        # step 1: define the mortality_label
        if next_visit.discharge_status not in [0, 1]:
            mortality_label = 0
        else:
            mortality_label = int(next_visit.discharge_status)

        # step 2: get code-based feature information
        conditions = visit.get_code_list(table="DIAGNOSES_ICD")
        procedures = visit.get_code_list(table="PROCEDURES_ICD")
        drugs = visit.get_code_list(table="PRESCRIPTIONS")
        drugs_full = visit.get_event_list(table="PRESCRIPTIONS")
        # if i == 0: print([d.attr_dict for d in drugs_full])
        # if i == 0: print(conditions)
        # if i == 0: print(procedures)
        # if i == 0: print(drugs)
        # TODO(botelho3) - add this datasource back in once we have full MIMIC-III dataset.
        # labevents = visit.get_code_list(table="LABEVENTS")

        # step 3: exclusion criteria: visits without condition, procedure, or drug
        if len(conditions) * len(procedures) == 0 * len(drugs_full) == 0: continue
        
        # step 3.5: build text lists from the ICD codes
        d_diag = mimic3base.get_text_lut("D_ICD_DIAGNOSES")
        d_proc = mimic3base.get_text_lut("D_ICD_PROCEDURES")
        # if i == 0: print(d_diag)
        # if i == 0: print(d_proc)
        # Index 0 is shortname, index 1 is longname.
        # print([str(cond) + ' ' + str(d_diag.get(cond)) for cond in conditions])
        # print(d_proc.get(procedures[0]))
        conditions_text = [d_diag.get(cond,("", ""))[1] for cond in conditions]
        procedures_text = [d_proc.get(proc,("", ""))[1] for proc in procedures]
        drugs_text = [' '.join([d['dname'], d['dtype'], d['dose'], d['route'], str(d['duration'])])
                      for d in [d.attr_dict for d in drugs_full]]
        # TODO(botelho3) - add the labevents data source back in once we have full MIMIC-III dataset.
        # labevents_text =
        
        # step 4: assemble the samples
        samples.append(
            {
                "visit_id": visit.visit_id,
                "patient_id": patient.patient_id,
                # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
                "conditions": conditions,
                "procedures": procedures,
                "conditions_text": conditions_text,
                "procedures_text": procedures_text,
                "drugs": drugs,
                "drugs_text": drugs_text,
                # "labevents": labevents,
                # "labevents_text": labevents_text
                "label": mortality_label,
            }
        )
    return samples


# mimic3sample = mimic3base.set_task(task_fn=drug_recommendation_mimic3_fn) # use default task
# train_ds, val_ds, test_ds = split_by_patient(mimic3sample, [0.8, 0.1, 0.1])

### DataLoaders and Collate

#### Bert Collate

In [ ]:
# def per_visit_collate_fn(data):
#     """
#     Collate the the list of samples into batches. For each patient, you need to pad the diagnosis
#         sequences to the sample shape (max # visits, len(freq_codes)). The padding infomation
#         is stored in `mask`.
    
#     Arguments:
#         data: a list of samples fetched from `CustomDataset`
        
#     Outputs:
#         x: a tensor of shape (# patiens, max # visits, len(freq_codes)) of type torch.float
#         masks: a tensor of shape (# patiens, max # visits) of type torch.bool
#         y: a tensor of shape (# patiens) of type torch.float
        
#     Note that you can obtains the list of diagnosis codes and the list of hf labels
#         using: `sequences, labels = zip(*data)`
#     """

#     sequences, labels = zip(*data)

#     y = torch.tensor(labels, dtype=torch.float)
    
#     num_patients = len(sequences)
#     num_visits = [len(patient) for patient in sequences]
#     max_num_visits = max(num_visits)
    
#     x = torch.zeros((num_patients, max_num_visits, len(freq_codes)), dtype=torch.float)    
#     for i_patient, patient in enumerate(sequences):
#         kMaxVisits = len(patient)
#         for j_visit, visit in enumerate(patient):
#             l = len(visit)
#             for code in visit:
#                 """
#                 TODO: 1. check if code is in freq_codes;
#                       2. obtain the code index using code2idx;
#                       3. set the correspoindg element in x to 1.
#                 """
#                 try:
#                     idx = code2idx[code]
#                     x[i_patient, j_visit, idx] = 1
#                 except KeyError as e:
#                     pass
    
#     masks = torch.sum(x, dim=-1) > 0
    
#     return x, masks, y

# def collate_fn(data):
#     """
#     TODO: Collate the the list of samples into batches. For each patient, you need to pad the diagnosis
#         sequences to the sample shape (max # visits, max # diagnosis codes). The padding infomation
#         is stored in `mask`.
    
#     Arguments:
#         data: a list of samples fetched from `CustomDataset`
        
#     Outputs:
#         x: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.long
#         masks: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.bool
#         rev_x: same as x but in reversed time. This will be used in our RNN model for masking 
#         rev_masks: same as mask but in reversed time. This will be used in our RNN model for masking
#         y: a tensor of shape (# patiens) of type torch.float
        
#     Note that you can obtains the list of diagnosis codes and the list of hf labels
#         using: `sequences, labels = zip(*data)`
#     """

#     sequences, labels = zip(*data)

#     y = torch.tensor(labels, dtype=torch.float)
    
#     num_patients = len(sequences)
#     num_visits = [len(patient) for patient in sequences]
#     num_codes = [len(visit) for patient in sequences for visit in patient]

#     max_num_visits = max(num_visits)
#     max_num_codes = max(num_codes)
    
#     x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
#     rev_x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
#     masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
#     rev_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
#     for i_patient, patient in enumerate(sequences):
#         kMaxVisits = len(patient)
#         for j_visit, visit in enumerate(patient):
#             """
#             TODO: update `x`, `rev_x`, `masks`, and `rev_masks`
#             """
#             l = len(visit)
#             x[i_patient, j_visit, 0:l] = torch.LongTensor(visit)
#             rev_x[i_patient, kMaxVisits-1-j_visit, 0:l] = torch.LongTensor(visit)
#             masks[i_patient, j_visit, 0:l] = 1
#             rev_masks[i_patient, kMaxVisits-1-j_visit, 0:l] = 1
#         # print(f"------ v: {kMaxVisits} ------")
#         # print(x[i_patient, :, ])
#         # print(rev_x[i_patient, :, ])
#         # print(masks[i_patient, :, ])
#         # print(rev_masks[i_patient, :, ])
    
#     return x, masks, rev_x, rev_masks, y


def bert_per_patient_collate_function(data):
    """
    Collates a tensor of (batch_size, seq_len, bert_emb_len) i.e. (32, <events per patient>, 768)
    Need to pad the second dimension to max(<variable_per_patient>).
    
    TODO: Collate the the list of samples into batches. For each patient, you need to pad the diagnosis
        sequences to the sample shape (max # visits, max # diagnosis codes). The padding infomation
        is stored in `mask`.
    
    Arguments:
        data: a list of samples fetched from `CustomDataset`
        
    Outputs:
        x: a tensor of shape (batch_size, max #conditions, max embedding size) of type torch.float
        masks: a tensor of shape (batch_size, max #conditions, max embedding_size) of type torch.bool
        rev_x: same as x but in reversed time. This will be used in our RNN model for masking 
        rev_masks: same as mask but in reversed time. This will be used in our RNN model for masking
        y: a tensor of shape (batch_size) of type torch.float
        
    Note that you can obtains the list of diagnosis codes and the list of hf labels
        using: `sequences, labels = zip(*data)`
    """
    # print(f"bert_per_patient_collate_function data[0] {data[0]}")
    sequences, labels = zip(*data)
    
    # Quick stats on the amount of memory in each batch.
    sizes = [t.element_size() * t.nelement() for t in sequences]
    print(f'BertCollate tensor sizes\n'
          f'cum_size:{sum(sizes) / 1.0e6} MB\n'
          f'sizes:{sizes}\n'
          f'{[tuple(x.shape) for x in sequences]}\n')
    
    # Convert output labels for each sample in batch to tensor.
    # shape: (batch_size, 1)
    y = torch.tensor(labels, dtype=torch.float)
   
    num_patients = len(sequences)
    num_events = [patient.shape[0] for patient in sequences]
    embedding_length = [patient.shape[1] for patient in sequences]

    max_num_events = max(num_events)
    max_embedding_length = max(embedding_length)
    
    x = torch.zeros((num_patients, max_num_events, max_embedding_length), dtype=torch.float)
    rev_x = torch.zeros((num_patients, max_num_events, max_embedding_length), dtype=torch.float)
    # Mask dimensions are 1 less than inputs.
    masks = torch.zeros((num_patients, max_num_events), dtype=torch.bool)
    rev_masks = torch.zeros((num_patients, max_num_events), dtype=torch.bool)
    for i_patient, patient in enumerate(sequences):
        # Patient (#events, 768)
        j_visits = patient.shape[0]
        # for j_visit, visit in enumerate(patient):
        """
        TODO: update `x`, `rev_x`, `masks`, and `rev_masks`
        """
        # l = len(visit)
        x[i_patient, :j_visits, :] = patient[:, :].unsqueeze(0)
        # The tensor is (seq_length, emb_size). Leave embeddings,
        # flip temporal order of code/event sequence.
        rev_x[i_patient, :j_visits, :] = torch.flip(patient, dims=[0]).unsqueeze(0)
        masks[i_patient, :j_visits] = 1
        rev_masks[i_patient, :j_visits] = 1
      
        # TODO(botelho3) - comment this out to reduce spew.
        # if i_patient == 0:
        #     print(f"------ p: {i_patient} ------")
        #     print(x[i_patient, :, :25])
        #     print(rev_x[i_patient, :, :25])
        #     print(masks[i_patient, :, :25])
        #     print(rev_masks[i_patient, :, :25])
    
    return x, masks, rev_x, rev_masks, y



#### Bert Load

In [ ]:
# Could use DistilBERT to decrease embedding time.
# https://huggingface.co/docs/transformers/model_doc/distilbert?highlight=distilberttokenizerfast#distilbert

class BertTextEmbedTransform(object):
    """Transform a sample's (a single visit's) text into 1 embedding vector.
    
    The embeddings of each text field are combined by embedding
    each separately then summing.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """
    # https://stackoverflow.com/questions/69517460/bert-get-sentence-embedding
    # https://huggingface.co/docs/transformers/internal/tokenization_utils#transformers.tokenization_utils_base.PreTrainedTokenizerBase
    # https://gmihaila.github.io/tutorial_notebooks/bert_inner_workings/

    def __init__(self, bert_model: Any, embedding_size: int, use_tokenizer_fast: bool):
        assert isinstance(embedding_size, (int, tuple))
        self.use_gpu = BERT_USE_GPU_
        self.cuda = GPU_STR_
        self.cache_dir = os.path.join(os.getcwd(), CACHE_DIR_)
        self.bert_config = BertConfig()
        # self.bert_config = AutoConfig.from_pretrained("bert-base-uncased")
        # self.bert_model = BertModel(self.bert_config).from_pretrained('bert-base-uncased', config=self.bert_config)
        # self.bert_model = AutoModel.from_pretrained('bert-base-uncased')
        with torch.no_grad():
            self.bert_model = BertModel(self.bert_config).from_pretrained('bert-base-uncased')
        if self.use_gpu:
            self.bert_model.to(self.cuda)
        self.bert_config = self.bert_model.config
        self.bert_model.eval()
        # We unfortunately can't put the tokenizer on the GPU.
        # https://stackoverflow.com/questions/66096703/running-huggingface-bert-tokenizer-on-gpu
        if use_tokenizer_fast:
            self.tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', model_max_length=100)
        else:
            self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', model_max_length=100)
    
    def _tokenize_text(self, text: str) -> str:
        tokenized_text = self.tokenizer.tokenize(text)
        return tokenized_text
    
    def _get_embeddings_of_sentences_with_mask(self, field, pad) -> torch.tensor:
        return self._get_embeddings_of_sentences(field[:pad])
        
    
    def _get_embeddings_of_sentences(self, sentences: List[str]) -> torch.tensor:
        # tokenized_sentences = [self.tokenizer.tokenize(t, padding=True) for t in sentences]
        # Tokenize the input sentence with attention masks.
        batch_enc = self.tokenizer.batch_encode_plus(sentences, padding=True,
                                return_attention_mask=True, return_length=True)
        # print(f'input sentence: {sentences[0]}\n'
        #       f'token sentence: {self.tokenizer.decode(batch_enc["input_ids"][0])}\n')
        batch_enc_tensor = batch_enc.convert_to_tensors(tensor_type=TensorType.PYTORCH)
        if self.use_gpu:
            batch_enc_tensor.to(self.cuda)
      
        # Run BERT model forward pass on tokenized input.
        with torch.no_grad():
            embeddings = self.bert_model(input_ids=batch_enc_tensor['input_ids'],
                                         attention_mask=batch_enc_tensor['attention_mask'],
                                         token_type_ids=batch_enc_tensor['token_type_ids'],
                                         # could turn off so we're faster
                                         output_attentions=True)
        # embeddings, _ = self.bert_model(**batch_enc)
        # print(f'embeddings:\n {dir(embeddings)}')
        #attention = encoded['attention_mask'].reshape((lhs.size()[0], lhs.size()[1], -1)).expand(-1, -1, 768)
       
        # These may be on GPU.
        return embeddings.last_hidden_state, embeddings.attentions 
    
    def __call__(self, sample):
        len_embeddings = (len(sample['conditions_text']) +
                          len(sample['procedures_text']) +
                          len(sample['drugs_text']))
      
        sample_text = []
        sample_masks = []
        if sample.get('conditions_text_pad'):
            pad = sample['conditions_text_pad']
            sample_text.extend(sample['conditions_text'][:pad])
        if sample.get('procedures_text_pad'):
            pad = sample['procedures_text_pad']
            sample_text.extend(sample['procedures_text'][:pad])
        if sample.get('drugs_text_pad'):
            pad = sample['drugs_text_pad']
            sample_text.extend(sample['drugs_text'][:pad])
           
        sample_embeddings, sample_attentions = self._get_embeddings_of_sentences(sample_text)
        
        # https://stackoverflow.com/questions/61323621/how-to-understand-hidden-states-of-the-returns-in-bertmodelhuggingface-transfo
        # https://datascience.stackexchange.com/questions/66207/what-is-purpose-of-the-cls-token-and-why-is-its-encoding-output-important
        # Take only the last hidden state embeddings from BERT.
        # We need to take index 0, because it's the CLS token prepended to our sentence.
        # The [CLS] represents the meaning of the whole sentence.
        # embeddings = torch.squeeze(sample_embeddings[:, -1, :], dim=1)
        embeddings = torch.squeeze(sample_embeddings[:, 0, :], dim=1)
        if self.use_gpu:
            assert(embeddings.device == torch.device('cuda:0'))
        else:
            assert(embeddings.device == torch.device('cpu'))
        if self.use_gpu:
            embeddings = embeddings.to('cpu')
        
        # We could multiply by attentions here:
        # attentions = torch.squeeze(sample_attentions[:, -1, :], dim=1)
        # embeddings = embeddings * attentions

        # The 1st dimension is seq length. The second dimension is embedding length of each sentence.
        assert(embeddings.shape[-1] == self.bert_config.hidden_size)
        assert(len(embeddings.shape) == 2)
        assert(embeddings.dtype == torch.float)
        
        # Return the ((model_inputs), label) 
        return (embeddings, sample['label'])
    
    
    
    
#     def __call__(self, sample):
#         condition_embeddings = [None] * len(sample['conditions_text'])
#         procedure_embeddings = [None] * len(sample['procedures_text'])
#         drug_embeddings = [None] * len(sample['drugs_text'])
       
#         if sample.get('conditions_text_pad'):
#             condition_embeddings, condition_attentions = self._get_embeddings_of_sentences_with_mask(
#                 sample['conditions_text'], sample['conditions_text_pad'])
#         else:
#             condition_embeddings, condition_attentions = self._get_embeddings_of_sentences(sample['conditions_text'])
#         # for cond in sample['conditions_text']:
#         #     #bert_model(cond)
#         #     condition_embeddings.append(cond)
#         #     pass
#         if sample.get('procedures_text_pad'):
#             procedure_embeddings, procedure_attentions = self._get_embeddings_of_sentences_with_mask(
#                 sample['procedures_text'], sample['procedures_text_pad'])
#         else:
#             procedure_embeddings, procedure_attentions = self._get_embeddings_of_sentences(sample['procedures_text'])
#         # for proc in sample['procedures_text']:
#         #     #bert_model(proc)
#         #     procedure_embeddings.append(proc)
#         #     pass
#         if (sample.get('drugs_text_pad')):
#             drug_embeddings, drug_attentions = self._get_embeddings_of_sentences_with_mask(
#                 sample['drugs_text'], sample['drugs_text_pad'])
#         else:
#             drug_embeddings, drug_attentions = self._get_embeddings_of_sentences(sample['drugs_text'])
#         # for drug in sample['drugs_text']:
#         #     #bert_model(drug)
#         #     procedure_embeddings.append(drug)
#         #     pass
        
#         # Take only the last hidden state embeddings from BERT.
#         condition_embeddings = torch.squeeze(condition_embeddings[:, -1, :], dim=1)
#         procedure_embeddings = torch.squeeze(procedure_embeddings[:, -1, :], dim=1)
#         drug_embeddings = torch.squeeze(drug_embeddings[:, -1, :], dim=1)
#         print(f'ce: {condition_embeddings.shape} '
#               f'pe: {procedure_embeddings.shape} '
#               f'de: {drug_embeddings.shape} ')
       
#         stacked = torch.cat([condition_embeddings, procedure_embeddings, drug_embeddings], dim=0)
        
#         # We don't want to normalize here because we need a sequence of embeddings for each sample. 
#         # normalize the final row (across columns)
#         # summed_embeddings = torch.sum(stacked, dim=0, keepdim=True)  # sum across rows
#         # summed_embeddings = torch.nn.functional.normalize(summed_embeddings, dim=1)
#         # print(summed_embeddings.shape)
#         # assert(summed_embeddings.shape == (1, self.bert_config.hidden_size))
       
#         # The 1st dimension is seq length. The second dimension is embedding length of each sentence.
#         assert(stacked.shape[-1] == self.bert_config.hidden_size)
#         assert(len(stacked.shape) == 2)
#         return (stacked, sample['label'])
    
    
class TextEmbedDataset(SampleDataset):
    '''The BERT text embedding process is very slow. We want to avoid it.
   
    To prevent re-processing of the same input cache the sample locally.
    Some suggestions here:
        https://stackoverflow.com/questions/61393613/pytorch-speed-up-data-loading.
        https://discuss.pytorch.org/t/best-practice-to-cache-the-entire-dataset-during-first-epoch/19608
        
    1. Preprocess and write the preprocessed text back out to disk.
    2. Cache the transform output in a hashtable. See functools.lru_cache().
    3. https://pytorch.org/data/main/ ?
    
    Some concerns related to num_workers > 1, i.e. multiprocessing enabled.
    See torch.save() to cache a tensor.
    '''
    
    def __init__(self, dataset: SampleDataset, transform=None):
        """Wraps a SampleEHRDataset with transforms.
        Arguments:
            dataset: dataset to transform
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        # self.dataset = dataset
        self.transform = transform
        self.transformed = [False for x in dataset]
        super().__init__([x for x in dataset])

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            assert(False)
            idx = idx.tolist()

        # Cache the transformed version of the data.
        if self.transform and not self.transformed[idx]:
            self.samples[idx] = self.transform(self.samples[idx])
            self.transformed[idx] = True

        return self.samples[idx]

In [ ]:
gc.collect()

# Create the transform that will take each sample (visit) in the dataset
# and convert the text description of the visit into a single embedding.
bert_xform = BertTextEmbedTransform(None, EMBEDDING_DIM_, use_tokenizer_fast=True)
BERT_EMBEDDING_SIZE = bert_xform.bert_config.hidden_size

# Mortality Task Datasets: set_task -> SampleEHRDataset(SampleBaseDataset)
# Add a transform to convert the visit into an embedding.
MORT_DEMB_CODE2IDX = {}
MORT_DEMB_CODE_COUNT = {}
task_fn = functools.partial(mortality_pred_task_per_patient, MORT_DEMB_CODE_COUNT)
mortality_demb_dataset = mimic3base.set_task(task_fn, task_name=mortality_pred_task_per_patient.__name__)
MORT_DEMB_CODE2IDX = {
    code: idx for idx, code in enumerate(sorted(MORT_DEMB_CODE_COUNT.keys()))
}
# Wrap the default pyHealth dataset class in our own wrapper. The wrapper takes each
# sample and applies BERT to xform text->pytorch.tensor.
mortality_demb_dataset = TextEmbedDataset(mortality_demb_dataset, transform=bert_xform)
mort_demb_train_ds, mort_demb_val_ds, mort_demb_test_ds = split_by_patient(mortality_demb_dataset, [0.8, 0.1, 0.1])

# create dataloaders (torch.data.DataLoader)
# mort_train_loader = get_dataloader(train_ds, batch_size=32, shuffle=True, collate_fn)
# mort_val_loader = get_dataloader(val_ds, batch_size=32, shuffle=False)
# mort_test_loader = get_dataloader(test_ds, batch_size=32, shuffle=False)

mort_demb_train_loader = DataLoader(
    mort_demb_train_ds,
    batch_size=BATCH_SIZE_,
    shuffle=SHUFFLE_,
    collate_fn=bert_per_patient_collate_function,
)
mort_demb_val_loader = DataLoader(
    mort_demb_val_ds,
    batch_size=BATCH_SIZE_,
    shuffle=SHUFFLE_,
    collate_fn=bert_per_patient_collate_function,
)
mort_demb_test_loader = DataLoader(
    mort_demb_test_ds,
    batch_size=BATCH_SIZE_,
    shuffle=SHUFFLE_,
    collate_fn=bert_per_patient_collate_function,
)



# from torch.utils.data import DataLoader

# def load_data(train_dataset, val_dataset, collate_fn):
    
#     '''
#     TODO: Implement this function to return the data loader for  train and validation dataset. 
#     Set batchsize to 32. Set `shuffle=True` only for train dataloader.
    
#     Arguments:
#         train dataset: train dataset of type `CustomDataset`
#         val dataset: validation dataset of type `CustomDataset`
#         collate_fn: collate function
        
#     Outputs:
#         train_loader, val_loader: train and validation dataloaders
    
#     Note that you need to pass the collate function to the data loader `collate_fn()`.
#     '''
    
#     batch_size = 32
#     train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
#     val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)
    


#     return train_loader, val_loader


In [ ]:
# Verify DataLoader properties.
# Quick test without running the whole RNN training process.

# from torch.utils.data import DataLoader

# loader = DataLoader(mort_, batch_size=10, collate_fn=collate_fn)
loader_iter = iter(mort_demb_train_loader)
# for _ in loader_iter:
#     pass
try:
    x, masks, rev_x, rev_masks, y = next(loader_iter)
except StopIteration as e:
    print(e)

assert x.dtype == torch.float
assert rev_x.dtype == torch.float
assert y.dtype == torch.float
assert masks.dtype == torch.bool
assert rev_masks.dtype == torch.bool

assert x.shape == (BATCH_SIZE_, 3, 105)
assert y.shape == (BATCH_SIZE_, 1)
assert masks.shape == (BATCH_SIZE_, 10, 3)

# assert x[0][0].sum() == 9
# assert masks[0].sum() == 2

#### CodeEmb Collate 

In [ ]:
# def code_emb_per_patient_collate_function(data):
#     """
#     UNUSED DONT PAY ATTENTION
#     TODO: Collate the the list of samples into batches. For each patient, you need to pad the diagnosis
#     sequences to the sample shape (max # visits, max # diagnosis codes). The padding infomation
#     is stored in `mask`.
                                   
    
#     Arguments:
#         data: a list of samples fetched from `CustomDataset`
        
#     Outputs:
#     x: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.long
#     masks: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.bool
#     rev_x: same as x but in reversed time. This will be used in our RNN model for masking
#     rev_masks: same as mask but in reversed time. This will be used in our RNN model for masking
#     y: a tensor of shape (# patiens) of type torch.float
        
#     Note that you can obtains the list of diagnosis codes and the list of hf labels\n",
#     using: `sequences, labels = zip(*data)`\n",
#     """
    
#     samples = data
#     print(f'code_emb_per_patient_collate_function data[0]:\n{data[0]}')
    
#     y = torch.tensor([s['label'] for s in samples], dtype=torch.float)
    
#     num_patients = len(samples)
#     num_events = [len(samples['conditions']) for patient in samples]
#     num_codes = [len(visit) for patient in sequences for visit in patient]

#     max_num_events = max(num_visits)
#     max_num_codes = max(num_codes)
    
#     x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
#     rev_x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
#     masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
#     rev_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
#     for i_patient, patient in enumerate(sequences):
#         kMaxVisits = len(patient)
#         for j_visit, visit in enumerate(patient):
#             """
#             TODO: update `x`, `rev_x`, `masks`, and `rev_masks`
#             """
#             l = len(visit)
#             x[i_patient, j_visit, 0:l] = torch.LongTensor(visit)
#             rev_x[i_patient, kMaxVisits-1-j_visit, 0:l] = torch.LongTensor(visit)
#             masks[i_patient, j_visit, 0:l] = 1
#             rev_masks[i_patient, kMaxVisits-1-j_visit, 0:l] = 1

#     return x, masks, rev_x, rev_masks, y

def code_emb_per_visit_collate_function(code2idx: Dict[str, int], data: List[Any]) -> Tuple[Any]:
    """
    TODO: Collate the the list of samples into batches. For each patient, you need to pad the diagnosis
    sequences to the sample shape (max # visits, max # diagnosis codes). The padding infomation
    is stored in `mask`.
                                   
    
    Arguments:
        data: a list of samples fetched from `CustomDataset`
        
    Outputs:
    x: a tensor of shape (# patients, max # visits, max # diagnosis codes) of type torch.long
    masks: a tensor of shape (# patients, max # visits, max # diagnosis codes) of type torch.bool
    rev_x: same as x but in reversed time. This will be used in our RNN model for masking
    rev_masks: same as mask but in reversed time. This will be used in our RNN model for masking
    y: a tensor of shape (# patiens) of type torch.float
        
    Note that you can obtains the list of diagnosis codes and the list of hf labels\n",
    using: `sequences, labels = zip(*data)`\n",
    """
    
    samples = data
    
    y = torch.tensor([s['label'] for s in samples], dtype=torch.float)
    
    num_patients = len(samples)
    num_visits = [patient['num_visits'] for patient in samples]
    num_codes = []
    for patient_idx, _ in enumerate(num_visits):
        num_codes.extend([len(visit) for visit in samples[patient_idx]['conditions']])
        
    # print(f'num samples: {len(samples)}')
    # print(f'num visits: {num_visits}')
    # print(f'num codes: {num_codes}')
    # print(f'code_emb_per_patient_collate_function data[0]:\n{data[0]}')
 
    max_num_visits = max(num_visits)
    max_num_codes = max(num_codes)
    # max_num_codes = len(MORTALITY_PER_VISIT_ICD_9_CODE2IDX_)
    assert(max_num_codes > 0)
    
    x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    rev_x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    rev_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    for i_patient, patient in enumerate(samples):
        nvisits = patient['num_visits']
        for j_visit in range(nvisits):
            """
            TODO: update `x`, `rev_x`, `masks`, and `rev_masks`
            """
            codes = patient['conditions'][j_visit] 
            indices = [code2idx[code] for code in codes]
            l = len(codes)
            if len(indices) < 1:
                print(f'No code indices after lookup')
                print(f'codes {codes}'+'\n'+f'indices{indices}')
                print(f'patient\n{patient}')
                assert(len(indices) >= 1)
            # for idx in indices:
            x[i_patient, j_visit, 0:l] = torch.tensor(indices)
            rev_x[i_patient, nvisits-1-j_visit, 0:l] = torch.tensor(indices)
            masks[i_patient, j_visit, 0:l] = 1
            rev_masks[i_patient, nvisits-1-j_visit, 0:l] = 1
        # if i_patient == 0:
        #     print(f"------ code_emb_per_visit_collate_function p: {i_patient} ------")
        #     print(x[i_patient, :, ])
        #     print(rev_x[i_patient, :, ])
        #     print(masks[i_patient, :, ])
        #     print(rev_masks[i_patient, :, ])
    
    return x, masks, rev_x, rev_masks, y

#### CodeEmb Load

In [ ]:
# Mortality Task Datasets: set_task -> SampleEHRDataset(SampleBaseDataset)
MORT_CEMB_CODE2IDX = {}
MORT_CEMB_CODE_COUNT = {}
mortality_cemb_ds = mimic3base.set_task(
    task_fn=functools.partial(mortality_pred_task_cemb, MORT_CEMB_CODE_COUNT),
    task_name=mortality_pred_task_cemb.__name__)
# The set_task(...) function iterates over all samples.
# Applying the task to each before returning a new dataset.
# Since all samples have been processed we have observed all codes and can
# build the code->index LUT.
MORT_CEMB_CODE2IDX = {
    code: idx for idx, code in enumerate(sorted(set(MORT_CEMB_CODE_COUNT.keys())))
}

# We need to provide the code->index LUT to the collate function.
wrap_code_emb_per_visit_collate_function = functools.partial(
    code_emb_per_visit_collate_function,
    MORT_CEMB_CODE2IDX)

# Split the dataset into train, val, and test.
mort_cemb_train_ds, mort_cemb_val_ds, mort_cemb_test_ds = split_by_patient(mortality_cemb_ds, [0.8, 0.1, 0.1])
mort_cemb_train_loader = DataLoader(
    mort_cemb_train_ds,
    batch_size=BATCH_SIZE_,
    shuffle=SHUFFLE_,
    collate_fn=wrap_code_emb_per_visit_collate_function,
)
mort_cemb_val_loader = DataLoader(
    mort_cemb_val_ds,
    batch_size=BATCH_SIZE_,
    shuffle=SHUFFLE_,
    collate_fn=wrap_code_emb_per_visit_collate_function,
)
mort_cemb_test_loader = DataLoader(
    mort_cemb_test_ds,
    batch_size=BATCH_SIZE_,
    shuffle=SHUFFLE_,
    collate_fn=wrap_code_emb_per_visit_collate_function,
)


In [ ]:
# Verify DataLoader properties.
# Quick test without running the whole RNN training process.

# from torch.utils.data import DataLoader

# loader = DataLoader(mort_, batch_size=10, collate_fn=collate_fn)
loader_iter = iter(mort_cemb_train_loader)
for _ in loader_iter:
    pass

try:
    x, masks, rev_x, rev_masks, y = next(loader_iter)
except StopIteration as e:
    print(e)

# assert x.dtype == torch.float
# assert rev_x.dtype == torch.float
# assert y.dtype == torch.float
# assert masks.dtype == torch.bool
# assert rev_masks.dtype == torch.bool

# assert x.shape == (BATCH_SIZE_, 3, 105)
# assert y.shape == (BATCH_SIZE_, 1)

## Model Training Loop

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, roc_curve, precision_recall_curve


def eval_model(model, val_loader):
    
    """
    TODO: evaluate the model.
    
    Arguments:
        model: the RNN model
        val_loader: validation dataloader
        
    Outputs:
        precision: overall precision score
        recall: overall recall score
        f1: overall f1 score
        roc_auc: overall roc_auc score
        
    Note that please pass all four arguments to the model so that we can use this function for both 
    models. (Use `model(x, masks, rev_x, rev_masks)`.)
        
    HINT: checkout https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics
    """
    
    model.eval()
    y_pred = torch.LongTensor()
    y_score = torch.Tensor()
    y_true = torch.LongTensor()
    model.eval()
    for x, masks, rev_x, rev_masks, y in val_loader:
        y_hat = model(x, masks, rev_x, rev_masks)
        y_score = torch.cat((y_score,  y_hat.detach().to('cpu')), dim=0)
        y_hat = (y_hat > 0.5).int()
        y_pred = torch.cat((y_pred,  y_hat.detach().to('cpu')), dim=0)
        y_true = torch.cat((y_true, y.detach().to('cpu')), dim=0)
    """
    TODO:
        Calculate precision, recall, f1, and roc auc scores.
        Use `average='binary'` for calculating precision, recall, and fscore.
    """
    # print(f'y_pred: {torch.sum(y_pred)}, y_true {torch.sum(y_true)}\n'
    #       f'y_score\n{y_score}\n'
    #       f'y_pred\n{y_pred}\n'
    #       f'y_true\n{y_true}\n')
    
    p, r, f, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    roc_auc = roc_auc_score(y_true, y_score)
    rcurve = roc_curve(y_true, y_score)
    precision_curve, recall_curve, thresholds = precision_recall_curve(y_true, y_score)
    
    

    # your code here
    return p, r, f, roc_auc, rcurve, precision_curve, recall_curve

In [ ]:
def train(model, train_loader, val_loader, n_epochs, criterion, optimizer):
    """
    TODO: train the model.
    
    Arguments:
        model: the RNN model
        train_loader: training dataloder
        val_loader: validation dataloader
        n_epochs: total number of epochs
        
    You need to call `eval_model()` at the end of each training epoch to see how well the model performs 
    on validation data.
        
    Note that please pass all four arguments to the model so that we can use this function for both 
    models. (Use `model(x, masks, rev_x, rev_masks)`.)
    """
    
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for x, masks, rev_x, rev_masks, y in train_loader:
            """
            TODO:
                1. zero grad
                2. model forward
                3. calculate loss
                4. loss backward
                5. optimizer step
            """
            optimizer.zero_grad()
            outputs = model(x, masks, rev_x, rev_masks)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            # your code here
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss))
        p, r, f, roc_auc, rcurve, _, _ = eval_model(model, val_loader)
        print('Epoch: {} \t Validation p: {:.2f}, r:{:.2f}, f: {:.2f}, roc_auc: {:.2f}'
              .format(epoch+1, p, r, f, roc_auc))

### Model: CodeEmb

In [ ]:
def cemb_sum_embeddings_with_mask(x, masks):
    '''
    Inputs:
        x: the embeddings of diagnosis sequence of shape
           (batch_size, # visits, # diagnosis codes, embedding_dim)
        masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)
    
    Outputs:
        sum_embeddings: the sum of embeddings of shape (batch_size, # visits, embedding_dim)
    '''
    tmp = torch.zeros(x.shape)
    m = torch.sum(masks, dim=2) > 0
    tmp[m, :, :] = 1
    tmp = x * tmp
    tmp = torch.sum(tmp, dim=2)
    
    a = x
    a[~masks] = 0
    tmp = torch.sum(a, dim=2)
    return tmp


def cemb_get_last_visit(hidden_states, masks):
    """
    TODO: obtain the hidden state for the last true visit (not padding visits)

    Arguments:
        hidden_states: the hidden states of each visit of shape (batch_size, # visits, embedding_dim=128)
        masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

    Outputs:
        last_hidden_state: the hidden state for the last true visit of shape (batch_size, embedding_dim=128)
        
    NOTE: DO NOT use for loop.
    
    HINT: First convert the mask to a vector of shape (batch_size,) containing the true visit length; 
          and then use this length vector as index to select the last visit.
    """
    m = torch.sum(masks, dim=2)
    m[m > 0] = 1
    m = torch.sum(m, dim=1)
    m[m > 0] = m[m > 0] - 1
   
    # (batch_size,)
    tmp1 = m
    # (batch_size,1,1)
    tmp = torch.reshape(m, (-1,1,1))
    # (batch_size,1,embedding_dim)
    tmp = tmp.expand(-1, -1, hidden_states.shape[2])
    # print(tmp)
    # print(tmp.shape)
    last_hidden_state = torch.gather(hidden_states, axis=1, index=tmp)
    # print(last_hidden_state.shape)
    last_hidden_state = torch.squeeze(last_hidden_state)
    
    # print(last_hidden_state[3, :])
    # print(torch.squeeze(hidden_states[3, tmp1[3], :]))
    assert(torch.equal(last_hidden_state[0, :], torch.squeeze(hidden_states[0, tmp1[0], :])))
    
    return last_hidden_state


class CembNaiveRNN(nn.Module):
    
    """
    TODO: implement the naive RNN model above.
    """
    
    def __init__(self, num_codes):
        super().__init__()
        """
        TODO: 
            1. Define the embedding layer using `nn.Embedding`. Set `embDimSize` to 128.
            2. Define the RNN using `nn.GRU()`; Set `hidden_size` to 128. Set `batch_first` to True.
            2. Define the RNN for the reverse direction using `nn.GRU()`;
               Set `hidden_size` to 128. Set `batch_first` to True.
            3. Define the linear layers using `nn.Linear()`; Set `in_features` to 256, and `out_features` to 1.
            4. Define the final activation layer using `nn.Sigmoid().

        Arguments:
            num_codes: total number of diagnosis codes
        """
        
        self.embedding = nn.Embedding(num_embeddings=num_codes, embedding_dim=128) 
        self.rnn = nn.GRU(input_size=128, hidden_size=128, batch_first=True)
        self.rev_rnn = nn.GRU(input_size=128, hidden_size=128, batch_first=True)
        self.fc = nn.Linear(in_features=256, out_features=1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x, masks, rev_x, rev_masks):
        """
        Arguments:
            x: the diagnosis sequence of shape (batch_size, # visits, # diagnosis codes)
            masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

        Outputs:
            probs: probabilities of shape (batch_size)
        """
        
        batch_size = x.shape[0]
        
        # 1. Pass the sequence through the embedding layer;
        # print(f"pre embedding: {x.shape}")
        e = self.embedding(x)
        # print(f"post embedding: {e.shape}")
        # 2. Sum the embeddings for each diagnosis code up for a visit of a patient.
        e = cemb_sum_embeddings_with_mask(e, masks)
        # print(f"post sum_embeddings_with_mask: {e.shape}")
        
        # 3. Pass the embeddings through the RNN layer;
        output, _ = self.rnn(e)
        # 4. Obtain the hidden state at the last visit.
        true_h_n = cemb_get_last_visit(output, masks)
        
        """
        TODO:
            5. Do the step 1-4 again for the reverse order (rev_x), and concatenate the hidden
               states for both directions;
        """
        # 1. Pass the sequence through the embedding layer;
        rev_e = self.embedding(rev_x)
        # 2. Sum the embeddings for each diagnosis code up for a visit of a patient.
        rev_e = cemb_sum_embeddings_with_mask(rev_e, rev_masks)
        
        # 3. Pass the embegginds through the RNN layer;
        output, _ = self.rnn(rev_e)
        # 4. Obtain the hidden state at the last visit.
        true_h_n_rev = cemb_get_last_visit(output, rev_masks)
        
        
        # 6. Pass the hidden state through the linear and activation layers.
        logits = self.fc(torch.cat([true_h_n, true_h_n_rev], 1))        
        probs = self.sigmoid(logits)
        return probs.view(batch_size)



In [ ]:
# load the model here
mort_code_naive_rnn = CembNaiveRNN(num_codes = len(MORT_CEMB_CODE2IDX))
mort_code_naive_rnn

### Model: DescEmb

RNN running and visualization

In [ ]:
def demb_sum_embeddings_with_mask(x, masks):
    '''
    Inputs:
        x: the embeddings of diagnosis sequence of shape (batch_size, # events(diag+proc+presc), embedding_dim)
        masks: the padding masks of shape (batch_size, # events(diag+proc+presc))
    Outputs:
        sum_embeddings: the sum of embeddings of shape (batch_size, embeddings_dim)
    '''
    # tmp = torch.zeros(x.shape)
    # m = torch.sum(masks, dim=1) > 0
    # tmp[m, :] = 1
    # tmp = x * tmp
    # tmp = torch.sum(tmp, dim=1)
    return x
    
    a = x
    a[~masks, :] = 0
    # tmp = torch.sum(a, dim=1)
    tmp = a
    return tmp


def demb_get_last_visit(hidden_states, masks):
    """
    TODO: obtain the hidden state for the last true visit (not padding visits)

    Arguments:
        hidden_states: the hidden states of each visit of shape
                       (batch_size, # events(diag+proc+presc), embedding_dim)
        masks: the padding masks of shape (batch_size, # events(diag+proc+presc))

    Outputs:
        last_hidden_state: the hidden state for the last true visit of shape (batch_size, embedding_dim)
        
    NOTE: DO NOT use for loop.
    
    HINT: First convert the mask to a vector of shape (batch_size,) containing the true visit length; 
          and then use this length vector as index to select the last visit.
    """
    m = torch.sum(masks, dim=1)
    # m[m > 0] = 1
    # m = torch.sum(m, dim=1)
    m[m > 0] = m[m > 0] - 1
    # print(f'selecting {m}')
    
    tmp1 = m
    tmp = torch.reshape(m, (-1,1,1))
    tmp = tmp.expand(-1, -1, hidden_states.shape[2])
    last_hidden_state = torch.gather(hidden_states, axis=1, index=tmp)
    last_hidden_state = torch.squeeze(last_hidden_state)
    # print(f'tmp.shape {tmp.shape}\n'
    #       f'last_hidden_state {last_hidden_state.shape}\n{last_hidden_state}')
    assert(torch.equal(last_hidden_state[0, :], torch.squeeze(hidden_states[0, tmp1[0], :])))
    
    return last_hidden_state


class DembNaiveRNN(nn.Module):
    
    """
    TODO: implement the naive RNN model above.
    """
    
    def __init__(self, bert_emb_size:int=BERT_EMBEDDING_SIZE):
        super().__init__()
        """
        TODO: 
            2. Define the RNN using `nn.GRU()`; Set `hidden_size` to 128. Set `batch_first` to True.
            2. Define the RNN for the reverse direction using `nn.GRU()`;
               Set `hidden_size` to 128. Set `batch_first` to True.
            3. Define the linear layers using `nn.Linear()`; Set `in_features` to 256, and `out_features` to 1.
            4. Define the final activation layer using `nn.Sigmoid().

        Arguments:
            num_codes: total number of diagnosis codes
        """
        self.bert_emb_size = bert_emb_size
        # self.embedding = nn.Embedding(num_embeddings=self.bert_emb_size, embedding_dim=128) 
        self.rnn = nn.GRU(input_size=self.bert_emb_size, hidden_size=128, batch_first=True)
        self.rev_rnn = nn.GRU(input_size=self.bert_emb_size, hidden_size=128, batch_first=True)
        self.fc = nn.Linear(in_features=256,out_features=1)
        self.sigmoid = nn.Sigmoid()
        # Contiguous memory.
        self.rnn.flatten_parameters()
        self.rev_rnn.flatten_parameters()
        
        
    def forward(self, x, masks, rev_x, rev_masks):
        """
        Arguments:
            x: the diagnosis sequence of shape (batch_size, #events(diag+proc+presc), embedding_dim)
            masks: the padding masks of shape (batch_size, #events(diag+proc+presc))

        Outputs:
            probs: probabilities of shape (batch_size)
        """
        
        batch_size = x.shape[0]
        
        # 2. Sum the embeddings for each diagnosis code up for a visit of a patient.
        x = demb_sum_embeddings_with_mask(x, masks)
        
        # 3. Pass the embeddings through the RNN layer;
        output, _ = self.rnn(x)
        # 4. Obtain the hidden state at the last visit.
        true_h_n = demb_get_last_visit(output, masks)
        
        """
        TODO:
            5. Do the step 1-4 again for the reverse order (rev_x), and concatenate the hidden
               states for both directions;
        """
        # xr = self.embedding(rev_x)
        xr = demb_sum_embeddings_with_mask(rev_x, rev_masks)
        routput, _ = self.rev_rnn(xr)
        true_h_n_rev = demb_get_last_visit(routput, rev_masks)
        
        # 6. Pass the hidden state through the linear and activation layers.
        logits =  self.fc(torch.cat([true_h_n, true_h_n_rev], 1))
        probs = self.sigmoid(logits)
        assert(probs.shape == (batch_size, 1))
        return probs.view(batch_size)
    
        
#     def forward_not_working(self, x, masks, rev_x, rev_masks):
#         """
#         Arguments:
#             x: the diagnosis sequence of shape (batch_size, #events(diag+proc+presc), embedding_dim)
#             masks: the padding masks of shape (batch_size, #events(diag+proc+presc))

#         Outputs:
#             probs: probabilities of shape (batch_size)
#         """
#         self.rnn.flatten_parameters()
#         self.rev_rnn.flatten_parameters()
        
#         batch_size = x.shape[0]
#         # e = self.embedding(x)
        
#         # 2. Sum the embeddings for each diagnosis code up for a visit of a patient.
#         x = demb_sum_embeddings_with_mask(x, masks)
        
#         # 3. Pass the embeddings through the RNN layer;
#         output, _ = self.rnn(x)
#         # 4. Obtain the hidden state at the last visit.
#         true_h_n = demb_get_last_visit(output, masks)
        
#         """
#         TODO:
#             5. Do the step 1-4 again for the reverse order (rev_x), and concatenate the hidden
#                states for both directions;
#         """
#         # xr = self.embedding(rev_x)
#         xr = demb_sum_embeddings_with_mask(rev_x, rev_masks)
#         routput, _ = self.rev_rnn(xr)
#         true_h_n_rev = demb_get_last_visit(routput, rev_masks)
        
#         # 6. Pass the hidden state through the linear and activation layers.
#         logits = self.fc(torch.cat([true_h_n, true_h_n_rev], 1))        
#         probs = self.sigmoid(logits)
#         assert(probs.shape == (batch_size, 1))
#         return probs.view(batch_size)
    

# load the model here
mort_desc_naive_rnn = DembNaiveRNN()
mort_desc_naive_rnn

### Train: CodeEmb

In [ ]:
print('Training CodeEmb.')

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(mort_code_naive_rnn.parameters(), lr=0.001)
# number of epochs to train the model
n_epochs =10 
train(mort_code_naive_rnn, mort_cemb_train_loader, mort_cemb_val_loader,
      n_epochs, criterion, optimizer)

In [ ]:
p, r, f, roc_auc, rcurve, prec_curve, rec_curve = eval_model(mort_code_naive_rnn, mort_cemb_val_loader)
print('Validation Set p: {:.2f}, r:{:.2f}, f: {:.2f}, roc_auc: {:.2f}'
      .format(p, r, f, roc_auc))

plt.plot(rcurve[0], rcurve[1])
plt.show()

plt.plot(rec_curve, prec_curve)
plt.show()

In [ ]:
p, r, f, roc_auc, rcurve,  prec_curve, rec_curve = eval_model(mort_code_naive_rnn, mort_cemb_test_loader)
print('Test Set p: {:.2f}, r:{:.2f}, f: {:.2f}, roc_auc: {:.2f}'
      .format(p, r, f, roc_auc))

plt.plot(rcurve[0], rcurve[1])
plt.show()

plt.plot(rec_curve, prec_curve)
plt.show()

### Train: DescEmb

In [ ]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(mort_desc_naive_rnn.parameters(), lr=0.001)

In [ ]:
# number of epochs to train the model
n_epochs = 5
train(mort_desc_naive_rnn, mort_demb_train_loader, mort_demb_val_loader,
      n_epochs, criterion, optimizer)

In [ ]:
p, r, f, roc_auc, rcurve,  prec_curve, rec_curve = eval_model(mort_desc_naive_rnn, mort_demb_val_loader)
print('Validation Set p: {:.2f}, r:{:.2f}, f: {:.2f}, roc_auc: {:.2f}'
      .format(p, r, f, roc_auc))

plt.plot(rcurve[0], rcurve[1])
plt.show()

plt.plot(rec_curve, prec_curve)
plt.show()

In [ ]:
p, r, f, roc_auc, rcurve,  prec_curve, rec_curve = eval_model(mort_desc_naive_rnn, mort_demb_test_loader)
print('Test Set p: {:.2f}, r:{:.2f}, f: {:.2f}, roc_auc: {:.2f}'
      .format(p, r, f, roc_auc))

plt.plot(rcurve[0], rcurve[1])
plt.show()

plt.plot(rec_curve, prec_curve)
plt.show()

### Load BERT Model

See instructions here: 
- https://pypi.org/project/pytorch-pretrained-bert/#examples
- https://neptune.ai/blog/how-to-code-bert-using-pytorch-tutorial
- Could also get it from pytorch transformers library: https://pytorch.org/hub/huggingface_pytorch-transformers/
- https://www.analyticsvidhya.com/blog/2021/05/all-you-need-to-know-about-bert/
- https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html - general text embedding.
- https://machinelearningmastery.com/handle-long-sequences-long-short-term-memory-recurrent-neural-networks/ handling long seq
- Encoder/Decoder training for embedding vocab.

In [ ]:
# https://neptune.ai/blog/how-to-code-bert-using-pytorch-tutorial
class BERTClassification(nn.Module):
    def __init__ (self):
        super(BERTClassification, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.bert_drop = nn.Dropout(0.4)
        self.out = nn.Linear(768, 1)

    def forward(self, ids, mask, token_type_ids):
        _, pooledOut = self.bert(ids, attention_mask = mask,
                                token_type_ids=token_type_ids)
        bertOut = self.bert_drop(pooledOut)
        output = self.out(bertOut)

        return output

In [ ]:

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


# TODO(botelho3)`bert-base-uncased` is big. Load `bert-tiny` instead from the filesystem?
# Model available at https://huggingface.co/prajjwal1/bert-tiny.
# model = BERT_CLASS.from_pretrained(PRE_TRAINED_MODEL_NAME_OR_PATH, cache_dir=None)

# Tokenized input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)

# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 8
tokenized_text[masked_index] = '[MASK]'
assert tokenized_text == ['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', '[MASK]', 'was', 'a', 'puppet', '##eer', '[SEP]']

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [ ]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
if USE_GPU_:
    tokens_tensor = tokens_tensor.to(GPU_STRING_)
    segments_tensors = segments_tensors.to(GPU_STRING_)
    model.to(GPU_STRING_)

# Predict all tokens
with torch.no_grad():
    predictions = model(tokens_tensor, segments_tensors)

# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
assert predicted_token == 'henson'

In [ ]:
# Use Dataset Transforms to overwrite the old Dataset with a new transformed dataset.
# Can either construct a new class BertDataset(Dataset): __init__(self, old_dataset)
# https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class
# Or can leverage transfomrs
# class Rescale(object): __init__(self) -> class BertTransform() __init__(bert_model), __call__(self, sample)